# Differential equation resolution

This notebook is inspired by the one in the Perceval documentation. It implements more ways to compute the resolution of a DE, including closer to the behavior of a real photonic chip ones.

## Initialisation

In [ ]:
from IPython import display
import perceval as pcvl
import perceval.lib.phys as phys
import numpy as np
from math import comb

from scipy.optimize import minimize

import time
import os
import multiprocessing
import pickle

import matplotlib.pyplot as plt
import matplotlib as mpl

# If not a notebook : from tqdm import tqdm
from tqdm.notebook import tqdm

import scipy.signal as signal

**True if we launch single computations during the notebook execution**

In [ ]:
# Set to True to launch single computations. Useful for testing particular settings
launch_single_computation = True

**Differential equation and parameters** : defined only once to make results comparable

In [ ]:
# Differential equation parameters
lambd = 8
kappa = 0.1

def F(u_prime, u, x):       # DE: F(u_prime, u, x) = 0
    # Must work with numpy arrays
    return (u_prime + lambd * u * (kappa + np.tan(lambd * x)))

In [ ]:
# Boundary condition (f(x_0)=f_0)
x_0 = 0
f_0 = 1

The solution of this differential equation is $f(x) = f_0  \exp(-\kappa \lambda x)  \cos(\lambda x)$ (must be changed in the documentation notebook).

In [ ]:
# Differential equation's exact solution - for comparison
def u(x):
    return np.exp(- kappa*lambd*x)*np.cos(lambd*x)

In [ ]:
# Modeling parameters
n_grid = 50    # number of grid points of the discretized differential equation
range_min = 0  # minimum of the interval on which we wish to approximate our function
range_max = 1  # maximum of the interval on which we wish to approximate our function
X = np.linspace(range_min, range_max, n_grid)  # Optimisation grid

In [ ]:
# Parameters of the quantum machine learning procedure
eta = 5                   # weight granted to the initial condition
a = 10                    # Approximate initial boundaries of the interval that the image of the trial function can cover
alpha = 0.0005            # Penality applied to the lambda coefficients

## Threshold detection

Here we create the function that will be used to compute the different probabilities in a generic case. It will be used for almost all the detection processes using a threshold detection.

The construction method uses the binary representation of a mode (1 if there is at least one photon, 0 else) to represent each mode by a single index. Equivalent modes share the same index.

With this detection method, we need $2^m$ weights if $N = m$. In fact, if the source is perfect, we should need only $2^m - 1$ weights as the 0 binary representation doesn't match any mode, but its easier to keep the same computation. 

If $N \ne m$, then more binary representations will not match anything (for example, if N = 2 and m = 4, the number 0111 will not appear). Thus, the computation could be quicker as it would lead to many $0*weight$. In fact, if $m = 2N$, the number of needed parameters is $\sum_{n = 0}^{n = N} {m\choose n} = 2^{m-1} + \frac{1}{2} {m\choose N}$.

In [ ]:
def create_threshold_indexes(m):
    """
    input : m[int] - the number of modes
    return : None
    
    Create an auxiliary list containing for each circuit output the index of the threshold probability list
    where the output probability must be summed
    
    Only used during the initialisation
    """
    
    # Here we create a list containing for each state the index in the threshold list that it corresponds to (i.e. where the probability will be summed).
    global threshold_indexes
    threshold_indexes = []
    
    #Necessary, else with an imperfect source, some states wouldn't be considered, even if the computation would not raise an error
    if not as_imperfect_source:
        
        sim = simulator_backend(pcvl.Matrix.random_unitary(m))
        sim.compile(input_state)
        
        iterator = sim.allstate_iterator(input_state)
    
    else:
        src = pcvl.Source(brightness=0, purity = purity) # The only goal here is to get all possible states
        
        processor = create_processor(src, phys.Unitary(pcvl.Matrix.random_unitary(m)), [1]*N+[0]*(m-N))
        iterator = get_all_processor_states(processor)
    
    for state in iterator:
        binary = []

        for mode in range(m):
            if state[mode] == 0:
                binary.append(0)
            else:
                binary.append(1)

        i = sum(val*(2**idx) for idx, val in enumerate(reversed(binary))) # Binary to decimal conversion
        
        if not method == "pseudo PNR" or state in B_useful_states:

            threshold_indexes.append(i)
    
    # Here we flatten the list so every index from 0 to sum^{N} (comb(m,n)) is represented, avoiding holes
    if m != N:
        mod_index = 0
        stop = max(threshold_indexes) + 1
        for cur_index in range(stop):
            l = [mod_index if item == cur_index else item for item in threshold_indexes]
            # If the new list is different, 
            if l != threshold_indexes:
                threshold_indexes = l
                mod_index += 1     

In [ ]:
def threshold_prob(all_prob):
    """
    input : all_prob[list] - the list of the probabilities given by the circuit
    output : np.array containing the threshold probabilities
    """
    
    sum_prob = np.zeros(fock_dim)
            
    for i in range(len(threshold_indexes)):
        sum_prob[threshold_indexes[i]] += all_prob[i]
        
    return sum_prob

## PNR computation

We try to upgrade our approximation by adding beam splitters at the end of the circuit to have a chance to separate the photons when there are several on the same mode. We double the number of modes to have empty modes at the beginning.

The computation now use both threshold detection and a bigger circuit to separate photons at the end.

We define a way to create our circuit separator that will come after the main circuit.

*Update* : This function is now only used to display, as a N mode circuit is now used in the computation; the beam splitters are applied in post-processing (see the computation paragraph below).

In [ ]:
# This function is only used to display the circuit, not in the calculation
def construct_separators(c):

    # First, we need to permutate our modes (it is equivalent to permuting lines and columns in our first matrix)
    perm =[]
    for mode in range(m):
        if mode < N:
            perm.append(2*mode)
        else:
            perm.append(2 * mode + 1 - 2 * N)

    c //= phys.PERM(perm)

    # We add BS between the first modes of the circuit and the unused modes
    for mode in range(N):
        c //= (2*mode, phys.BS())
    
    return c

**Computation**

The method used to calculate all the probabilities for a $2N$ modes beam splitting circuit $\mathcal B$ receiving inputs from a $N$ modes circuit $\mathcal A$ is the following:

If two modes $(2k, 2k+1)$ from $\mathcal B$ have $n_1$ and $n_2$ photons, then the only possibility to have this output is that the $k^{th}$ mode from $\mathcal A$ had $n_1 + n_2$ photons.

Since we want to be able to consider only the probability list (easier for computation than considering the states), we can create an auxiliary list called *pi_factors* containing for each output mode of $\mathcal B$ the index of the corresponding output mode in $\mathcal A$ and the probability to get this $\mathcal B$ state from the $\mathcal A$ state.

In [ ]:
def create_pi_factors(N):
    """
    input : N[int] - the number of photons
    Returns : None
    
    Creates a list allowing to get the probabilities for a 2N modes circuit from a N modes circuit using beam splitters
    
    Support for reflexivity has not been fully implemented yet
    """
    # We first create a matrix containing the probabilities to get i photons in the 2k mode of B from n photons in the k mode of A
    probs = np.zeros((N+1,N+1))
    
    
    # We can only have up to N photons in one A mode
    for n in range(N+1):  
        for i in range(n+1):
            # It is much faster to use a direct formula, and more precise in term of float operation
            probs[n, i] = comb(n, i) * R ** i * (1-R) ** (n-i)
    
    
    # Now we can create our list
    if not as_imperfect_source:
        
        sB = simulator_backend(pcvl.Matrix.random_unitary(2*N))
        sA = simulator_backend(pcvl.Matrix.random_unitary(N))
        
        input_state = pcvl.BasicState([N, 0]) # We input N photons, the goal is to have all the outputs with N photons
        
        B_iterator = sB.allstate_iterator(input_state)
        A_iterator = list(sA.allstate_iterator(input_state))
              
    else:
        src = pcvl.Source(brightness=0, purity = purity) # The only goal here is to get all possible states

        A_processor = create_processor(src, phys.Unitary(pcvl.Matrix.random_unitary(N)), [1]*N)
        A_iterator = get_all_processor_states(A_processor)
        
        B_processor = create_processor(src, phys.Unitary(pcvl.Matrix.random_unitary(2*N)), [1]*N + [0]*N)
        B_iterator = get_all_processor_states(B_processor)
        
        
        
    global pi_factors
    pi_factors = []
    
    
    global B_useful_states
    B_useful_states = []

    # B_state has at most N photons
    for B_state in B_iterator:      
        # If we do a post selection and all_photons are not seen, we must not register that state
        if (method != "post selection" or all_photons_seen(B_state)):
        
            A_state = []
            p = 1
            must_register = True
            for k in range(0, 2*N, 2):
                i = B_state[k]                
                n = i + B_state[k+1]

                
                """
                This part needs to be adapted to different values of R
                It is true only for R = .5
                
                It had been implemented because for R = .5, we always have some probabilities that are equal, leading to bad results.
                For other values of R, this still has to be evaluated.
                """ 
                # If we can see photons in only one of the two modes, it is equivalent to consider only the (n, 0) one with twice the probability
                if i == 0 and n != 0:
                    p = 2
                if i == n and i != 0: # ie B_state[k+1] == 0 and the state is not [0,0]
                    must_register = False
             
                p *= probs[n, i]
                A_state.append(n)

                
                
            if must_register:
                B_useful_states.append(B_state)
                A_state = pcvl.BasicState(A_state)
                index = 0

                
                # We now need to find the index in the A circuit that corresponds to this state
                for state in A_iterator:
                    if state == A_state:
                        pi_factors.append((index, p))
                        # We can break since it can only appear once
                        break

                    index += 1    

Next, we just have to use this prepared list to compute the probabilities given the list of probabilities from $\mathcal A$. Note these are generic probabilities, so if we want to apply a threshold, it must be done after this function.

In [ ]:
def calc_all_probs(A_probs):  
    probs = []
    
    for (A_state_index, prob) in pi_factors:
        probs.append(A_probs[A_state_index] * prob)
        
    return np.array(probs)

We try to speed up the computation by avoiding the use of an intermediate list in case of PNR.

In [ ]:
def calc_PNR_probs(A_probs):
    sum_prob = np.zeros(fock_dim)    
    
    for i in range(len(threshold_indexes)):
        sum_prob[threshold_indexes[i]] += A_probs[pi_factors[i][0]] * pi_factors[i][1]
        
    return sum_prob

**Post selection**

We will use a different way to compute the probabilities compared to in the non-selecting process. In fact, because we only consider the states having at most 1 photon for each output, each binary representation will only point to 1 state. Thus, we just have to remember for each ponderation the corresponding state, allowing us not to visit all the states. This gives us ${m \choose N}$ vectors to verify. This quantity doesn't change if the source isn't perfect.

Because we only put useful states in the *pi_factors* list, we do not have to apply a threshold computation on this (I'm not sure if this is true when we select states having at least $n$ photons with $n \ne N$).

In [ ]:
def all_photons_seen(state):
    # Count how many modes have exactly 1 photon
    mode_count = sum([1 for mode in state if mode == 1])
    
    return (mode_count == N)

## Imperfect source

We will now try our different learning processes with an imperfect source. Since the source is the same for all input mode, we first define a function to define easily.

In [ ]:
def create_processor(source, circuit, input_modes):
    # input_modes[i] == 1 if the ith mode has a source
    inputs = dict()
    for i in range(len(input_modes)):
        if input_modes[i] == 1:
            inputs[i] = source
    
    return pcvl.Processor(inputs, circuit)

In [ ]:
def get_all_processor_probabilities(processor):
    # Returns a numpy array with the probabilities of the output states
    _, output_distribution = processor.run(simulator_backend)
    
    items = output_distribution.items()
    
    all_prob = np.array([prob for _, prob in items])
        
    return all_prob

In [ ]:
def get_all_processor_states(processor):
    # Returns a numpy array with the probabilities of the output states
    _, output_distribution = processor.run(simulator_backend)
    
    items = output_distribution.items()
    
    # state is a StateVector, so we need to extract the only BasicState that is here
    all_states = [state[0] for state, _ in items]
    
    return all_states

## Imprecision

It is impossible to be as precise as we want on a real chip. Computation show that the imprecision is $\sqrt N \Delta \theta$. We will use a round effect to take this into account, as the imprecision is always the same for a given voltage input.

An imprecision of $\Delta \theta$ will also be introduced in the definition of x (which is concretely an angle for a phase shifter).

In [ ]:
def round_imprecision(x, imprecision):
    # Can take numpy arrays
    if imprecision == 0:
        return x
    else:
        return imprecision * np.round(x / imprecision)

In [ ]:
def imprecise_matrix(parameters, delta_theta):
    U = pcvl.Matrix.random_unitary(N, parameters)
    
    # sqrt(2) because our imprecision is in norm, while round acts on real and imag values
    total_imprecision = delta_theta / np.sqrt(2)  

    
    U = round_imprecision(U, total_imprecision) 

    # We need to make the noised matrix unitary
    return pcvl.Matrix.random_unitary(N, parameters = np.concatenate((np.real(U).reshape(N2), np.imag(U).reshape(N2))))

Since we now have a huge step to calculate the gradient for the different parameters (we need to see a difference to optimise), we try to add a second step in which we optimise the ponderations while leaving the parameters unchanged. The best would be to do both at the same time, but it would lead to too many iterations.

After some tries, it is so efficient that I decided to use that second step first and whatever the parameters are (this computation has been moved to the Computation section).

## Samples

Since we cannot evaluate the probabilities with a 100% accuracy, we need to simulate samples that we will observe on a real chip from the distribution that is computed from Perceval. If the number of samples is *huge* (more than 100), then the second version of the function must be used, due to computation time (just change the order of the cells when executing all). For the second version, the sampling is estimated through the variance in the number of states got.

The probability to get $k$ times a state with a probability $p$ for one experiment follows the binomial law $\mathfrak{B} (n_{photons}, p)$. Thus, the standard deviation is $\sqrt{n_{photons} p (1-p)}$, so the probability distribution that we get has a standard deviation of $\sqrt{\frac{p (1-p)}{n_{photons}}}$.

In [ ]:
# First version : true sampling, very long due to the finite difference method to evaluate the gradient. Time in O(N_samples * len(probs))
def get_samples(N_samples, probs):
    
    # No sampling, get the full distribution
    if N_samples == 0:
        return probs
    
    rng = np.random.default_rng()
    
    # Generates N_samples indexes with the distribution probs    
    choosen = rng.choice(len(probs), N_samples, p = probs)
    
    observed = np.array([sum(1 for i in range(N_samples) if choosen[i] == n) for n in range(len(probs))])
        
    return observed / N_samples

In [ ]:
# Second version : distribution sampling, constant time whatever N_samples is
def get_samples(N_samples, probs):
    
    # No sampling, get the full distribution
    if N_samples == 0:
        return probs
    
    rng = np.random.normal(scale = np.sqrt(probs * (1 - probs) / N_samples))
    
     
    observed = probs + rng
    
    # Renormalise the probabilities
    observed /= sum(observed)
        
    return observed

In the case we are conserving the global distribution probability, we need to retrieve the samples for each probability.

In [ ]:
def get_all_samples(N_samples, all_probs):
    # La construction a la même structure que celle de get_final_probs
    probs = []
    
    for prob in all_probs:
        probs.append(get_samples(N_samples, prob))
        
    return probs

We try to avoid problems in the computation due to the noise by applying a noise filter. This noise filter adapt to our grid.

In [ ]:
def noise_filter(Y):
    # For now, we use default parameter. This must be changed later with global parameters
    order = 2
    Wn = 0.17
      
    b, t = signal.butter(order, Wn)     
    return signal.filtfilt(b, t, Y)

## Photon loss

We now consider that photons can be lost in the circuit or not detected. We represent this loss as a probability $p$ for each photon not to be detected at the end, without repercussion on the internal behaviour of the chip.

If we consider a mode $i$ having $n_i$ photons after the computation, thus the probability to get $k_i$ photons on this mode after the loss is ${n_i \choose k_i} p^{n_i-k_i}(1-p)^{k_i}$.

Then, the probability to get a given state is $ p^{N - k} (1-p)^{k} \prod^{m}_{i = 1} {n_i \choose k_i}$ with $k = \sum k_i$.

We can now create a list containing for each outcome of the imperfect device the list of the indexes of the interesting outcomes of the perfect device with their associated probability. If we are interested in a threshold detection, we can already apply it, speeding up the computation.

*In the case we are performing a pseudo PNR or a post selection method, the probability of losing the photons on the $N*N$ circuit is conserved on the $2N*2N$ circuit.* Thus, we can perform this computation before simulating the $2N*2N$ circuit.

**Proof:** let's consider one mode of the $N*N$ circuit $\mathcal A$. If there is $n$ photons on this mode, the probability to lose $k$ photons is ${n \choose k} q_{\mathcal A}^{n-k}(1-q_{\mathcal A})^{k}$ with $q_{\mathcal A} = 1-p_{\mathcal A}$ if there is no beam splitter after. 

Now let's consider the two associated modes after the beam splitter (circuit $\mathcal B$) with a reflexivity $R$. If the mode $\mathbb 1$ gets $n_1$ photons, then the mode $\mathbb 2$ gets $n - n_1$ photons. By the same way, if the mode $\mathbb 1$ loses $k_1$ photons, then the mode $\mathbb 2$ loses $k - k_1$ photons. 

Furthermore, in a general case, the probability that the mode $\mathbb 1$ loses $k_1$ photons knowing it has $n_1$ photons and the mode $\mathbb 2$ loses $k_2$ photons knowing it has $n_2$ photons is $$
\mathcal P ((\mathbb{1}.k =  k_1) \cap (\mathbb{2}.k =  k_2) | \mathbb{1}.n = n_1, \mathbb{2}.n = n_2) = {n_1 \choose k_1} q_{\mathcal B}^{n_1-k_1}(1-q_{\mathcal B})^{k_1} * {n_2 \choose k_2} q_{\mathcal B}^{n_2-k_2}(1-q_{\mathcal B})^{k_2}
$$

where $q_{\mathcal B}$ is the known photon loss. The goal here is to express $q_{\mathcal B}$ as a function of $q_{\mathcal A}$. The previous equation rewrites in our case as 
$$
\mathcal P ((\mathbb{1}.k = k_1) \cap (\mathbb{2}.k =  k - k_1) | \mathbb{1}.n = n_1, \mathbb{2}.n = n - n_1) = {n_1 \choose k_1} q_{\mathcal B}^{n_1-k_1}(1-q_{\mathcal B})^{k_1} * {n - n_1 \choose k - k_1} q_{\mathcal B}^{n - n_1 - k + k_1}(1-q_{\mathcal B})^{k - k_1}
$$

Now we have for the total loss 
$$
\mathcal P (\mathbb{1}.k + \mathbb{2}.k = k | \mathbb{1}.n = n_1, \mathbb{2}.n = n - n_1) = \sum_{k_1 = 0}^{n_1} \mathcal P ((\mathbb{1}.k = k_1) \cap (\mathbb{2}.k =  k - k_1) | \mathbb{1}.n = n_1, \mathbb{2}.n = n - n_1) 
$$

Then setting $n_1$ as an unknown gives
$$
\mathcal P (\mathbb{1}.k + \mathbb{2}.k = k | \mathbb{1}.n + \mathbb{2}.n = n) = \sum_{n_1 = 0}^n \sum_{k_1 = 0}^{n_1} \mathcal P ((\mathbb{1}.k = k_1) \cap (\mathbb{2}.k =  k - k_1) | \mathbb{1}.n = n_1, \mathbb{2}.n = n - n_1)
* \mathcal P (\mathbb{1}.n = n_1 | \mathbb{1}.n + \mathbb{2}.n = n)
$$

But as there is a beam splitter with no photon on the second mode, this gives us $\mathcal P (\mathbb{1}.n = n_1 | \mathbb{1}.n + \mathbb{2}.n = n) = {n \choose n_1} R^{n_1} (1-R)^{n - n_1}$

Finally, we get 
$$
\mathcal P (\mathbb{1}.k + \mathbb{2}.k = k | \mathbb{1}.n + \mathbb{2}.n = n) = \sum_{n_1 = 0}^n \sum_{k_1 = 0}^{n_1} {n_1 \choose k_1} q_{\mathcal B}^{n_1-k_1}(1-q_{\mathcal B})^{k_1} * {n - n_1 \choose k - k_1} q_{\mathcal B}^{n - n_1 - k + k_1}(1-q_{\mathcal B})^{k - k_1} * {n \choose n_1} R^{n_1} (1-R)^{n - n_1}
$$

$$
\mathcal P (\mathbb{1}.k + \mathbb{2}.k = k | \mathbb{1}.n + \mathbb{2}.n = n) = q_{\mathcal B}^{n - k} (1-q_{\mathcal B})^{k} \sum_{n_1 = 0}^n \sum_{k_1 = 0}^{n_1} {n_1 \choose k_1} {n - n_1 \choose k - k_1} {n \choose n_1} R^{n_1} (1-R)^{n - n_1}
$$

With the following code (a written proof could be provided later), we observe that 
$$
\sum_{n_1 = 0}^n \sum_{k_1 = 0}^{n_1} {n_1 \choose k_1} {n - n_1 \choose k - k_1} {n \choose n_1} R^{n_1} (1-R)^{n - n_1} = {n \choose k}
$$

Finally, this gives us the equivalence of the model between a loss at the end of $\mathcal A$ and at the end of $\mathcal B$.
$$
{n \choose k} q_{\mathcal A}^{n-k}(1-q_{\mathcal A})^{k} = {n \choose k} q_{\mathcal B}^{n - k} (1-q_{\mathcal B})^{k}
$$

with $q_{\mathcal A} = q_{\mathcal B}$

In [ ]:
N = 8
k = 5
R = .4

def my_comb(n, k):
    if k<0:
        return 0
    else:
        return comb(n, k)

print("left side =", sum(comb(n1, k1) * my_comb(N - n1, k - k1) * comb(N, n1) * R**n1 * (1-R)**(N-n1) for n1 in range(N+1) for k1 in range(n1+1)))

print("right side =", comb(N, k))

In [ ]:
def create_photon_loss(m, N):
    # First, we compute all the necessary powers of photon_loss
    p_powers = [1]
    q_powers = [1]
    for n in range(N):
        p_powers.append(p_powers[-1] * photon_loss)
        q_powers.append(q_powers[-1] * (1 -photon_loss))
        

    # Now we can create our list
    
    # Creates an imperfect source
    source = pcvl.Source(brightness= 1 - photon_loss)
    
    # We create an input state iterator
    if not use_imperfect_source:
        sB = simulator_backend(phys.Unitary(pcvl.Matrix.random_unitary(m)))
        input_state = pcvl.BasicState([N, 0]) # We input N photons, the goal is to have all the output with N photons
        
        # list is necessary to reset the iterator on each iteration
        B_iterator = list(sB.allstate_iterator(input_state))
    
    else:
        B_processor = create_processor(source, phys.Unitary(pcvl.Matrix.random_unitary(m)), [1]*N + [0] * (m-N))
        B_iterator = get_all_processor_states(B_processor)
    
    # In any case, the target will have the same states than if the source is imperfect
    src = pcvl.Source(brightness=0, purity = purity) # The only goal here is to get all possible states 
    A_processor = create_processor(src, phys.Unitary(pcvl.Matrix.random_unitary(m)), [1]*N + [0] * (m-N))
    A_iterator = get_all_processor_states(A_processor)
    
    
    global p_loss_list
    p_loss_list = []
    
    
    for A_state in A_iterator:
        used_states = []
        
        k = A_state.n
        
        index = 0
            
        for B_state in B_iterator:
            
            # Check whether the target state can be reached from the B state
            if np.all([A_state[i] <= B_state[i] for i in range(m)]):
                n = B_state.n   # Useful if the main source is not perfect
                
                prob = p_powers[n - k] * q_powers[k]
                prob *= np.prod([comb(B_state[i], A_state[i]) for i in range(m)])
                
                used_states.append((index, prob))
                
            index += 1
        
        p_loss_list.append(used_states.copy())

In [ ]:
def compute_photon_loss(probs):
    final_probs = np.zeros(len(p_loss_list))
    
    for i in range(len(p_loss_list)):
        final_probs[i] = sum(p_loss_list[i][j][1] * probs[p_loss_list[i][j][0]] for j in range(len(p_loss_list[i])))
        
    return final_probs    

# Generic functions

Because the computation is similar in every case, we define generic functions that will be defined with a *method*, so we can compute all the tries more easily

## Auxiliary functions

In [ ]:
def tuple_to_dict(tpl):
    # tpl is of the form (key, value, ...)
    
    # We use a default dictionnary for when a parameter doesn't appear
    dictionnary = {"method" : None, "N" : 0, "imprecision" : 0, "samples" : 0, "photon loss" : 0, "brightness" : 1, "purity" : 1}
    
    for k in range(0, len(tpl), 2):
        dictionnary[tpl[k]] = tpl[k+1]
        
    if "R" not in dictionnary.keys() and dictionnary["method"] in ["pseudo PNR", "post selection"]:
        dictionnary["R"] = 0.5
        
    return dictionnary

## Initialisation

In [ ]:
def get_fock_dim(m, N, method):
    if method == "real solving" or method == "fixed ponderations":
        if as_imperfect_source: # We must then consider all cases when less than N photons are detected
            return sum([comb(n + m - 1, n) for n in range(N+1)])
        return comb(N + m - 1, N)
    
    elif method == "threshold":
        return 2**m
    
    elif method == "pseudo PNR":
        #return 2**(m-1) + comb(m, N) // 2  # Used before the elimination of the 0s of threshold_indexes
        return max(threshold_indexes) + 1
    
    elif method == "post selection":
        #return comb(m, N)
        return len(pi_factors)

In [ ]:
def initiate(param_dict, display = True):   
    # display:bool - if the circuit has to be displayed
    
    
    # First part : retrieve the parameters from the dictionnary and set them as global variables 
    global method
    method = param_dict["method"]
        
    global N
    global m
    N = param_dict['N']              # Number of photons
    
    global R
    R = .5   
    if method == "pseudo PNR" or method == "post selection":
        m = 2 * N
        if "R" in param_dict.keys():
            R = param_dict["R"]
    else:
        m = N
    
    # N2 will be used frequently
    global N2
    N2 = N ** 2
    
    
    
    global use_imperfect_source
    use_imperfect_source = False
    
    global brightness
    if "brightness" in param_dict.keys():   
        brightness = param_dict["brightness"]
        if brightness != 1:
            use_imperfect_source = True
    else:
        brightness = 1
    
    global purity
    if "purity" in param_dict.keys():   
        purity = param_dict["purity"]
        if purity != 1:
            use_imperfect_source = True
    else:
        purity = 1
            
            
    if use_imperfect_source:
        global source
        source = pcvl.Source(brightness=brightness, purity=purity)
    
    
      
    global imprecision
    if "imprecision" in param_dict.keys():     
        imprecision = param_dict["imprecision"]
    else:
        imprecision = 0
        
    
    global samples
    if "samples" in param_dict.keys():     
        samples = param_dict["samples"]
    else:
        samples = 0
     
    
    global as_imperfect_source   # Means that the output can have photon loss
    as_imperfect_source = use_imperfect_source
                
    global photon_loss
    if "photon loss" in param_dict.keys():     
        photon_loss = param_dict["photon loss"]
        if photon_loss != 0:
            as_imperfect_source = True
    else:
        photon_loss = 0

    ##############################################################################################################
    # Second part : initialise the computation variables
    
    # Input state with N photons and m modes
    global input_state
    input_state = pcvl.BasicState([1]*N+[0]*(m-N))
    
    global simulator_backend
    simulator_backend = pcvl.BackendFactory().get_backend("SLOS")
    
    # Create a simulator of the good size, useless if there is an imperfect source (another object will be used)
    if not use_imperfect_source:           
        global s1
        s1 = simulator_backend(pcvl.Matrix.random_unitary(N))
        s1.compile(input_state)
    
    
    
    # If necessary, compute the calculus auxiliary list(s)
    if photon_loss != 0:
        create_photon_loss(N, N) # used on the NxN circuit, equivalent to simulation on the m*m circuit as shown before
    if method == "pseudo PNR" or method == "post selection":
        create_pi_factors(N)
    if method == "threshold" or method == "pseudo PNR":
        create_threshold_indexes(m)
        
        
    # Change here the value of alpha (must be parameterized)
    global alpha
    if samples == 0:
        alpha = 0
    else:
        alpha = 0

    
    global fock_dim
    global lambda_random
    fock_dim = get_fock_dim(m, N, method)
    # lambda coefficients for all the possible outputs
    lambda_random = np.random.rand(fock_dim)
    
    # Normalise the distribution to avoid random impossible solving parameters
    lambda_random = a * (lambda_random - np.mean(lambda_random)) / np.std(lambda_random)
    
    # dx serves for the numerical differentiation of f
    global dx
    # dx = (range_max-range_min) / 1000         # Used for first order finite difference

    dx = (range_max-range_min)/(n_grid - 1)         # Used for second order finite difference, faster and more precise
    
 
    
    "Haar unitary parameters"
    # number of parameters used for the two universal interferometers (2*N**2 per interferometer) as initial guess
    global parameters
    parameters = np.random.normal(size=4*N2)
     
    # Make the initial guess unitary if there is imprecision (useful for relative step size)
    if imprecision != 0:
        A = pcvl.Matrix.random_unitary(N, parameters[:2 * N2])
        parameters[:2 * N2] = np.concatenate((np.real(A).reshape((N2,)), np.imag(A).reshape((N2,))))
        
        A = pcvl.Matrix.random_unitary(N, parameters[2 * N2:4 * N2])
        parameters[2 * N2:4 * N2] = np.concatenate((np.real(A).reshape((N2,)), np.imag(A).reshape((N2,))))
      
    
    if method == "real solving":
        parameters = np.concatenate((parameters, lambda_random))


    # Just display the circuit, not used after
    if display:
        px = pcvl.P("px")
        c = (pcvl.Circuit(m)
             // phys.Unitary(pcvl.Matrix.random_unitary(N, parameters[:2 * N2]), name="W1")
             // (0, phys.PS(px))
             // phys.Unitary(pcvl.Matrix.random_unitary(N, parameters[2 * N2:4 * N2]), name="W2"))

        if method == "pseudo PNR" or method == "post selection":
            c = construct_separators(c)
  
        pcvl.pdisplay(c)
    
    # It was temporary for the display and list creation, but it is needed that they become equal for what is next
    m = N
    
    input_state = pcvl.BasicState([1]*N+[0]*(m-N))

In [ ]:
def initiate_optimisation():
    # Variables set to record and monitor the optimisation
    np.random.seed(int(10000*time.time()+os.getpid()) & 0xffffffff) #not necessary here, but when we run our program on several cores, this line allows us to generate different unit matrices at the same time.
    global computation_count
    computation_count = 0
    
    global current_loss
    current_loss = 0
    
    global start_time
    start_time = time.time()
    
    global loss_evolution
    loss_evolution = []
    
    global iteration_count
    iteration_count = 1  # callback is at the end of each iteration in case of convergence not reached
    
    global total_computation_count
    total_computation_count = 0
        
    if use_pbar_and_display:
        global pbar
        pbar = tqdm()

## The computation

A generic function to compute the probabilities.

In [ ]:
def get_probs(circuit, input_state, method):

    if not use_imperfect_source:
        U = circuit.compute_unitary(use_symbolic=False)
        s1.U = U
        
        probs = s1.all_prob(input_state)
    else:
        process = create_processor(source, circuit, [1]*N+[0]*(m-N))
        probs = get_all_processor_probabilities(process)
        
        
    if photon_loss != 0:
        probs = compute_photon_loss(probs)
    
  
    # Simulates a 2m circuit if needed
    if method =="post selection":
        probs = calc_all_probs(probs)
        
    if method == "threshold":
        probs = threshold_prob(probs)
        
    if method == "pseudo PNR":
        probs = calc_PNR_probs(probs)
    
    # random observations according to the distribution
    probs = get_samples(samples, probs)
    
    return probs

In [ ]:
def calc(circuit, input_state, coefs, method):
    probs = get_probs(circuit, input_state, method)
    
    return np.sum(np.multiply(probs, coefs))        

Loss function in the case we are modifying the matrix parameters.

In [ ]:
def computation(params):
    global current_loss
    global computation_count
    "compute the loss function when the matrix parameters are being optimized"
    computation_count += 1

    coefs = lambda_random  # coefficients of the M observable
    # initial condition with the two universal interferometers and the phase shift in the middle
    
    if imprecision == 0:
        U_1 = pcvl.Matrix.random_unitary(N, params[:2 * N2])
        U_2 = pcvl.Matrix.random_unitary(N, params[2 * N2:4 * N2])
        X_used = X
        
   
    else:
        U_1 = imprecise_matrix(params[:2 * N2], imprecision)
        U_2 = imprecise_matrix(params[2 * N2:4 * N2], imprecision)
        X_used = round_imprecision(X, imprecision)
        

    # Circuit creation
    px = pcvl.P("px")
    c = (phys.Unitary(U_2)
         // (0, phys.PS(px)) 
         // phys.Unitary(U_1))

    
    px.set_value(round_imprecision(x_0, imprecision))
    f_theta_0 = calc(c, input_state, coefs, method)

     
    # boundary condition given a weight eta
    loss = eta * (f_theta_0 - f_0) ** 2 * n_grid
    
        
    # Warning : Y[0] is before the domain we are interested in (used for differentiation), the domain begins at Y[1]
    Y = np.zeros(n_grid + 2)
    
    # Small optimisation working if x_0 == range_min
    if x_0 == range_min:
        Y[1] = f_theta_0
        assigned = 1
    else:
        assigned = 0
        
    
    px.set_value(round_imprecision(range_min - dx, imprecision))
    Y[0] = calc(c, input_state, coefs, method)
    
    
    
    for i in range(assigned, n_grid):
        x = X_used[i]

        px.set_value(x)
        Y[i + 1] = calc(c, input_state, coefs, method)
    
    
    # Y_prime[0] is the beginning of the domain /!\ not the same for Y
    px.set_value(round_imprecision(range_max + dx, imprecision)) 
    Y[n_grid + 1] = calc(c, input_state, coefs, method)
    
    
    # Here is one problem that must be solved. 
    if samples != 0:
        Y = noise_filter(Y)
    
    
    # Or we can compute Y_prime by finite difference using Y and not dx
    # The approximation is far better with a second order scheme
    # This could be the fastest way to compute the derivative (or at least an approximation)

    Y_prime = (Y[2:] - Y[:-2])/(2*dx)
    
    # This method is apparently the fastest to calculate the L2 norm squared
    loss += np.sum((F(Y_prime, Y[1:-1], X))**2)
        
    
    # Displayed
    current_loss = loss / n_grid
    
    # Useless for this part of the optimisation, but useful to keep the same loss value
    loss = current_loss + alpha * np.sum(coefs ** 2)
    return loss 

Functions used to calculate the loss when minimizing via the ponderations.

In [ ]:
def get_final_probs(params):
    """
    Gather all the probabilities at the points of the grid in a single object    
    """
    
    U_1 = imprecise_matrix(params[:2 * N2], imprecision)
    U_2 = imprecise_matrix(params[2 * N2:4 * N2], imprecision)
    X_used = round_imprecision(X, imprecision)

    px = pcvl.P("px")
    c = (phys.Unitary(U_2)
         // (0, phys.PS(px)) 
         // phys.Unitary(U_1))
    

    # probs[0] corresponds to x0, probs[1] to -dx, ..., probs[-1] to range_max + dx
    probs = []
    
    px.set_value(round_imprecision(x_0, imprecision))
    probs.append(get_probs(c, input_state, method))
    
    px.set_value(round_imprecision(range_min - dx, imprecision))
    probs.append(get_probs(c, input_state, method))
    
    if x_0 == range_min:
        probs.append(probs[0])
        assigned = 1
    else:
        assigned = 0
    
    for i in range(assigned, n_grid):
        x = X_used[i]

        px.set_value(x)
        probs.append(get_probs(c, input_state, method))

        
    px.set_value(round_imprecision(range_max + dx, imprecision))
    probs.append(get_probs(c, input_state, method))
 
    return probs

In [ ]:
def loss_from_ponderations(coefs):
    """
    This is the loss function when optimizing the ponderations. Its result is the same than for the previous function
    """
    global current_loss
    f_theta_0 = np.sum(np.multiply(coefs, probs[0]))
    
    loss = eta * (f_theta_0 - f_0) ** 2 * n_grid
    
    Y = np.array([np.sum(np.multiply(coefs, probs[i])) for i in range(1, len(probs))])
    
    
    # Here is one problem that must be solved. 
    if samples != 0:
        Y = noise_filter(Y)
    
    Y_prime = (Y[2:] - Y[:-2])/(2*dx)
    
    # This method is apparently the fastest to calculate the L2 norm squared
    loss += np.sum((F(Y_prime, Y[1:-1], X))**2)
        
    # Displayed
    current_loss = loss / n_grid
    
    loss = current_loss + alpha * np.sum(coefs ** 2)
    return loss    

The function to launch the computations.

In [ ]:
def callbackF(parameters):
    """callback function called by scipy.optimize.minimize allowing to monitor progress"""
    global current_loss
    global computation_count
    global loss_evolution
    global start_time
    global iteration_count
    
    if step == "circuit":
        iteration_count += 1    
        
    now = time.time()
    if use_pbar_and_display:
        pbar.set_description("N= %d Loss: %0.5f #computations: %d elapsed: %0.5f step: " %
                             (N, current_loss, computation_count, now-start_time) + step)
        pbar.update(1)
    loss_evolution.append((current_loss, now-start_time))
    computation_count = 0
    start_time = now
    
    if samples != 0 and not iteration_count % 5 and step == "ponderations":
        # The distribution doesn't change here, we just want to avoid bias due to sampling
        global probs
        probs = get_all_samples(samples, all_probs)

In [ ]:
def run_optimisation(param):
    """
    input : param[tuple[(key1, value1, key2, value2...)]]
    
    output : list containing:
    - the parameters to get the matrix
    - the ponderations
    - a list containing the loss and the computation time at each iteration
    - the number of iterations during the matrix parameters optimization part
    - the total number of distribution evaluation. Multiply this by the number of samples to know of many samples are required in total.
    - the final loss. It might differ from the last loss of the previous list as if the circuit optimisation is worse than the ponderation part, results are saved from the ponderation part.
    """
    param_dict = tuple_to_dict(param)

    global all_results
    initiate_optimisation()
    initiate(param_dict, display = use_pbar_and_display)

    global lambda_random
    global parameters

    global step

    global probs

    conv_crit = 1e-2
    max_iter = 5

    global all_probs

    i = 1
    while True:

        all_probs = get_final_probs(parameters)

        step = "ponderations"            
        # Does nothing if samples = 0
        probs = get_all_samples(samples, all_probs)
        if samples != 0:
            # Just add some boundaries to avoid huge values of lambda. However, BFGS doesn't support constraints. Impact still has to be evaluated
            res2 = minimize(loss_from_ponderations, lambda_random, bounds = len(lambda_random)*[(-a, a)], callback=callbackF, method='L-BFGS-B', options={'gtol': 1E-5})
        else:
            res2 = minimize(loss_from_ponderations, lambda_random, callback=callbackF, method='BFGS', options={'gtol': 1E-5})

        lambda_random = res2.x
        error_ponderations = res2.fun - alpha * np.sum(lambda_random ** 2)

        step = "circuit"

        if samples != 0:
            # This option can work sometimes 
            res = minimize(computation, parameters, callback=callbackF, method='COBYLA', options = {'rhobeg' : 1/(20 * np.sqrt(2*N))}) 


        elif imprecision != 0:
            # Use epsilon relatively to the error
            res = minimize(computation, parameters, callback=callbackF, method='BFGS', options={'gtol': 1E-2, 'eps' : 2 * imprecision})          

        else:
            # Use default epsilon = 1.4901161193847656e-08
            res = minimize(computation, parameters, callback=callbackF, method='BFGS', options={'gtol': 1E-2})

        error_circuit = res.fun  - alpha * np.sum(lambda_random ** 2)

        if error_ponderations < error_circuit:
            # We don't change the parameters then
            final_loss = error_ponderations
            break

        parameters = res.x

        if abs(error_ponderations - error_circuit) / error_ponderations < conv_crit or i == max_iter:
            # In any case, the result is the one containing the circuit parameters
            final_loss = error_circuit
            break   

        i += 1


    if method == "real solving":
        lambda_random = res.x[4 * N2:]

    # Save the results
    return [parameters.copy(), lambda_random.copy(), loss_evolution.copy(), iteration_count, total_computation_count, final_loss]

The launch of the computations itself

In [ ]:
if launch_single_computation:
    # Initialisation of the results
    # Can also be used to throw the previous results

    all_results = dict()

In [ ]:
if launch_single_computation:
    
    use_pbar_and_display = True
    
    '''
    Needed parameters are  and "N"
    ("method", str) possible choices of method are "real solving", "fixed ponderations", "pseudo PNR", "threshold", and "post selection"
    ("N", int) The number of photons. PNR and post selection use twice as much modes, other method have m = N
       
    Other parameters can be specified with 
    ("brightness", float[0:1]) 1 - the probability that a photon is not emitted at the source. Too slow to be used for now. Default : 1
    ("purity", float[0:1]) 1 - the probability that 2 photons are emitted at the source. Too slow to be used for now. Default : 1
    ("photon loss", float[0:1]) The probability that a photon is not detected. Default : 0
    ("imprecision", float) The imprecision on the angles of your interferometer. Default : 0
    ("samples", unsigned int) The number of samples used for evaluating the probability distribution. 0 gives the true distribution. Default : 0
    '''
    
    params = [("method", "post selection", "N", 6)]
    for param in params:
        if not param in all_results.keys():
            all_results[param] = []
    
    
    for param in params:     
        all_results[param].append(run_optimisation(param))

Just verify that the results are the computed ones. Also useful to see how much noise we can expect.

In [ ]:
print(max(abs(lambda_random)))
print(min(abs(lambda_random)))

print(all_results[params[0]][-1][5])

## Results

We first plot the graph of the solutions

In [ ]:
def plot_solution(X_plot, optim_params, lambda_random, param_dict):
    # You have to run initiate before this one when you change your parameters 
    
    N = param_dict["N"]
    N2 = N ** 2
    method = param_dict["method"]
    imprecision = param_dict["imprecision"]
    Y = []
    
    U_1 = imprecise_matrix(optim_params[:2 * N2], imprecision)
    U_2 = imprecise_matrix(optim_params[2 * N2:4 * N2], imprecision)

    X_used = round_imprecision(X_plot, imprecision)
    
    px = pcvl.P("x")
    
    c = phys.Unitary(U_2) // (0, phys.PS(px)) // phys.Unitary(U_1)
    
    
    for x in X_used:
        px.set_value(x)       
        f_theta = calc(c, input_state, lambda_random, method)
        Y.append(f_theta)
    
    #p = plt.plot(X_plot, Y, label=get_legend(param_dict))
      
    if samples != 0:
        Y = noise_filter(Y)

        #plt.plot(X_plot, Y, p[-1].get_color(), label = "filtfilt")
    
    p = plt.plot(X_plot, Y, label=get_legend(param_dict))

In [ ]:
def get_legend(param_dict, excluded = None):
    """
    Creates the legend for your set of parameters. If one parameter is the default one, it doesn't appear.
    """  
    default_values = tuple_to_dict(())
    
    if excluded != "method" and excluded != "N":
        legend = f'{param_dict["method"]} with {param_dict["N"]} photons'
    
    if excluded == "method":
        legend = f'{param_dict["N"]} photons'
        
    if excluded == "N":
        legend = f'{param_dict["method"]}'
           
    key = "imprecision"
    if excluded != key and param_dict[key] != default_values[key]:
        legend += f' and {key} of {param_dict[key]} rad'
    
    key = "samples"
    if excluded != key and param_dict[key] != default_values[key]:
        legend += f' and {param_dict[key]} {key}'
    
    for key in ["photon loss", "brightness", "purity"]:
        if excluded != key and param_dict[key] != default_values[key]:
            legend += f' and a {key} of {param_dict[key] * 100}%'
        
    key = "R"
    if excluded != key and key in param_dict.keys() and param_dict[key] != .5:
        legend += f'and a reflexivity of {param_dict[key]}'

    return legend

In [ ]:
if launch_single_computation:
    # Plot all approximations for given criteria

    # Change the parameters to see here, leave empty for all the simulations
    criteria = {"N" : 6}

    X_plot = np.linspace(range_min, range_max, 200)

    # Change the plot size
    default_figsize = mpl.rcParamsDefault['figure.figsize']
    mpl.rcParams['figure.figsize'] = [2 * value for value in default_figsize]
    

    for key in all_results.keys():
        key_dict = tuple_to_dict(key)
        if criteria.items() <= key_dict.items():
            for solution in all_results[key]:    
                #key_dict["imprecision"] = 0           # turn this on to plot as if there was no imprecision     
                initiate(key_dict, display = False)

                plot_solution(X_plot, solution[0], solution[1], key_dict)

    plt.plot(X_plot, u(X_plot), 'k', label='Analytical solution')
    
    #plt.title(str(criteria)[1:-1])
    plt.legend()
    plt.show()

Then we plot the final loss for the different methods and number of photons, as well as the computation time and the total number of iterations. (beware some of the graphs are in log scale)

In [ ]:
# Functions to retrieve the data
def get_losses(results):
    losses = dict()
    
    for params in results.keys():
        losses[params] = []
        for solution in results[params]:
            losses[params].append(solution[5])
            
    return losses


def get_calc_times(results):
    calc_times = dict()
    
    for params in results.keys():
        calc_times[params] = []
        for solution in results[params]:
            calc_times[params].append(np.sum(np.array(solution[2])  / 60, axis = 0)[1])    # In minutes
            
    return calc_times


def get_it_number(results):
    it_number = dict()
    
    for params in results.keys():
        it_number[params] = []
        for solution in results[params]:
            it_number[params].append(solution[3])
            
    return it_number


def get_computation_count(results):
    computation_count = dict()
    
    for params in results.keys():
        computation_count[params] = []
        for solution in results[params]:
            computation_count[params].append(solution[4])
            
    return computation_count

In [ ]:
def acceptable(dict_key, allow_dict, forbid_dict):
    if forbid_dict == None:
        return allow_dict.items() <= dict_key.items()
    else:
        return allow_dict.items() <= dict_key.items() and not forbid_dict.items() <= dict_key.items()

# Computes the mean and standard deviation
def plot_statistics(results, y_axis, x_axis, allow_dict = dict(), forbid_dict = None, y_scale = "default"):
    """
    x_axis takes any of the parameters
    y_axis accepts one of "loss", "computation time", "iteration number", "computation count"
    """
    # In any case, Y is a dictionnary of lists
    if y_axis == "loss":
        Y = get_losses(results)
    elif y_axis == "computation time":
        Y = get_calc_times(results)
    elif y_axis == "iteration number":
        Y = get_it_number(results)
    elif y_axis == "computation count":
        Y = get_computation_count(results)
        
        
        
    if x_axis == "photons":
        x_axis_used = "N"
    else: 
        x_axis_used = x_axis
        
    # Useful for log scale, must be changed
    if forbid_dict == None and x_axis == "imprecision":
        forbid_dict={"imprecision" : 0}
        
        
    if x_axis_used == "N" or x_axis_used == "samples":
        # Only integer ticks will be displayed
        ax = plt.figure().gca()
        ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True)) 
    
    
    # number of keys = number of tuples of parameters
    nb_params = []
    all_params = [0]
    for key in results.keys():
        x = tuple_to_dict(key)[x_axis_used]
        key_dict = tuple_to_dict(key)
        
        if acceptable(key_dict, allow_dict, forbid_dict) and len(Y[key]):       
            all_params.append(x)
            nb_params.append(sum(1 for key2 in results.keys() if tuple_to_dict(key2)[x_axis_used] == x and acceptable(tuple_to_dict(key2), allow_dict, forbid_dict)))
    
    all_params = np.array(all_params)
    all_params = np.sort(np.unique(all_params))
    
    n_params = max(nb_params)
    if len(all_params) >= 2:
        one_param_size = min(all_params[1:] - all_params[:-1])
    else:
        one_param_size = 1
    
    graph_step = one_param_size * (0.8 + 0.1/n_params)/n_params
    width = one_param_size * 0.8 - (n_params - 1) * graph_step
    
    visited = []
    X_ticks = []
    
    # Main loop
    i = 0
    for key in results.keys(): 
        key_dict = tuple_to_dict(key) 
        if not key in visited and acceptable(key_dict, allow_dict, forbid_dict):
            
            # We can now visit all keys having the same parameters other than x_axis
            key_dict.pop(x_axis_used)
            
            X = []       # Created "by hand" because the order is important
            
            means = []
            stds = []
            
            for cur_key in results.keys():
                cur_key_dict = tuple_to_dict(cur_key)
                
                if key_dict.items() <= cur_key_dict.items() and acceptable(cur_key_dict, allow_dict, forbid_dict):
                    visited.append(cur_key)
                    
                    if len(Y[cur_key]): # If there are computations
                        means.append(np.mean(Y[cur_key]))
                        stds.append(np.std(Y[cur_key]))
                        
                        X.append(cur_key_dict[x_axis_used])
                        X_ticks.append(cur_key_dict[x_axis_used])
                        
            shift = i * graph_step + width/2 - 0.4 * one_param_size
            plt.bar(np.array(X) + shift, means, yerr = stds, label = get_legend(key_dict, excluded = x_axis_used), width = width);
            i += 1           

    
    plt.xlabel(x_axis);
    if y_scale == "default":
        if y_axis == "loss" or y_axis == "computation time" or y_axis == "computation count":
            plt.yscale("log")
    else:
        plt.yscale(y_scale)
        
    #plt.xscale("log")  # Doesn't work; the width is bigger on the left
        
    plt.xticks(np.unique(X_ticks))
    if y_axis == "computation time":
        plt.ylabel(y_axis + " (min)")
    else:
        plt.ylabel(y_axis);
    plt.title("means and standard deviation of " + y_axis);
    plt.grid(axis = 'y', linestyle = '--', linewidth = 0.7)
    plt.legend();

    plt.show()

In [ ]:
if launch_single_computation:

    # Small graphs
    default_figsize = mpl.rcParamsDefault['figure.figsize']
    mpl.rcParams['figure.figsize'] = [1.5 * value for value in default_figsize]
    
    # Change here to see other things
    x_axis = "photon loss"

    plot_statistics(all_results, "loss", x_axis)
    plot_statistics(all_results, "computation time", x_axis)
    plot_statistics(all_results, "iteration number", x_axis)
    plot_statistics(all_results, "computation count", x_axis)

We are also interested to see what part of the loss is due to the initial condition and what part is from the respect of the DE.

In [ ]:
def get_loss_parts(optim_params, lambda_random, total_loss, param_dict):
    method = param_dict["method"]
    imprecision = param_dict["imprecision"]
    U_1 = imprecise_matrix(optim_params[:2 * N2], imprecision)
    U_2 = imprecise_matrix(optim_params[2 * N2:4 * N2], imprecision)
    px = pcvl.P("x")
    
    c = phys.Unitary(U_2) // (0, phys.PS(px)) // phys.Unitary(U_1)
           
    px.set_value(round_imprecision(x_0, imprecision))         
     
    f_theta_0 = calc(c, input_state, lambda_random, method)
    
    loss_boundary = eta * (f_theta_0 - f_0) ** 2
    
    return loss_boundary, total_loss - loss_boundary




def get_all_loss_parts(results):
    losses = get_losses(results)
    detailled_losses = dict()
    
    global method
    for params in results.keys():
        param_dict = tuple_to_dict(params)
            
        detailled_losses[params] = []
        
        for solution in results[params]: 
            
            initiate(param_dict, display = False)

            detailled_losses[params].append(get_loss_parts(solution[0], solution[1], solution[5], param_dict))
            
    return detailled_losses

In [ ]:
def get_photon_range(results, selection_dict):
    aux = [tuple_to_dict(key)['N'] for key in results.keys() if selection_dict.items() <= tuple_to_dict(key).items()]
    
    n_min = min(aux)
    n_max = max(aux)
    
    return n_min, n_max


def get_best_solutions(results):

    best_solutions = dict()

    for key in results.keys():
        if results[key] != []:
            best_solutions[key] = min((l, i) for (i, l) in enumerate(solution[5] for solution in results[key]))
            
    return best_solutions


In [ ]:
def plot_decomposed_loss(results):

    visited = []
    
    best_solutions = get_best_solutions(results)

    for key in results.keys():
        dict_key = tuple_to_dict(key)

        if not key in visited:
            key_without_N = dict_key.copy()
            key_without_N.pop('N')
            
            n_min, n_max = get_photon_range(results, key_without_N)
            nphotons = range(n_min, n_max + 1)

            Y = np.zeros((n_max - n_min + 1, 2)) 
            for i in range(n_max - n_min + 1):

                initiate(dict_key, display = False)

                N_index = key.index('N') + 1
                cur_key = list(key)
                cur_key[N_index] = nphotons[i]
                cur_key = tuple(cur_key)

                visited.append(cur_key)
                
                if len(results[cur_key]):
                    best_index = best_solutions[cur_key][1]
                    solution = results[cur_key][best_index]

                    initiate(tuple_to_dict(cur_key), display = False)

                    Y[i, :] = get_loss_parts(solution[0], solution[1], solution[5], tuple_to_dict(cur_key))


            boundary_loss = Y[:, 0]
            DE_loss = Y[:, 1]

            plt.bar(np.array(nphotons) - 0.2, boundary_loss, label = "boundary loss", width=0.3);
            plt.bar(np.array(nphotons) + 0.2, DE_loss, label = "Differential equation loss", width=0.3);

            plt.xlabel("nb of photons");
            plt.ylabel("Loss");
            plt.yscale("log")            
            plt.title("Separated loss for " + get_legend(dict_key));
            plt.legend();
            plt.show()

In [ ]:
if launch_single_computation:
    # May take some time to plot all the graphs if there are many different sets of parameters
    plot_decomposed_loss(all_results)

# Multiple runs

To run multiple (hundreds) times the simulation to make statistics, a dataframe should be more convenient than a dictionnary. However, since the size of the lambda parameters is not fixed, we cannot use pandas, which doesn't support lists as values. We store our results in files, with one file containing one experiment and the name of the folder telling the parameters.

In [ ]:
def add_parameter(directory, key, default_values, param_dict):
    
    if key in param_dict.keys() and param_dict[key] != default_values[key]:
        directory += f"_{key}:{param_dict[key]}"
    
    return directory


def get_directory_name(param_dict):
    """ Function used to always have the keys in the same order in the folder's name;
    otherwise, N:_method:_ would lead to a different key than method:_N: while being equivalent
    
    A key doesn't appear if its value is the default one. This is to allow the adding of new keys.
    
    Chosen order : method, N, brightness, purity, photon loss, imprecision, samples
    """
    
    directory = f"./Results/N:{param_dict['N']}_method:{param_dict['method']}"
    
    default_values = tuple_to_dict(())
    
    
    keys = ['brightness', 'purity', 'photon loss', 'imprecision', 'samples']
    for key in keys:
        directory = add_parameter(directory, key, default_values, param_dict)
        
    
    return directory 

In [ ]:
# The name of the parameters is necessarily weird to put them as global variables after
def run_optimisation_parallel(param):
    # Run a single optimisation using the parameters 
    solution = run_optimisation(param)
    
    # We now dump the result into a file, creating a folder if necessary
    param_dict = tuple_to_dict(param)
    directory = get_directory_name(param_dict)
    
    try:
        os.mkdir(directory)
    except:
        pass
    
    i = len(os.listdir(directory)) + 1
    with open(directory + f"/{i}.pkl", 'wb') as f:
        pickle.dump(solution, f)

In [ ]:
'''
Launch the computation.
The results will stack up with the older ones.
Consider running the cell two places above if you want to clean up the old results.

Sometimes, the progress bar acts weird and stops two iterations before the end, even if all is finished.
'''


# Redefined here in case we just want to run some cases but keep the previous results
methods = ["fixed ponderations", "pseudo PNR", "post selection"]
nphotons = [n for n in range(2, 7)]
photon_losses = [p_loss for p_loss in np.arange(0, 0.55, 0.05)]
    
one_iteration = [("method", method, "N", n, "photon loss", p_loss) for method in methods for n in nphotons for p_loss in photon_losses]
        
# Defines how much computations for each tuple of parameters will be performed
iteration_number = 50


# Allow computations to be done without interruption at the end of each cycle as if would be if it was done with a for loop
all_parameters_iterated = iteration_number * one_iteration

imax = len(all_parameters_iterated)

# Toggles display of the circuit and run by run off to avoid spam. Instead, we show how much computations have been done
use_pbar_and_display = False
pbar = tqdm(total = imax)


# It's useless to allocate more processes than the number of computations
process_count = min(os.cpu_count(), imax)


def parallel_run():
    # The with command is used so the pool is automatically closed at the end of the computation
    # By default, all the processors of the computer are used
    with multiprocessing.Pool(process_count) as pool:

        results = [pool.apply_async(run_optimisation_parallel, parameters) for parameters in all_parameters_iterated]
        
        pbar.update(1)
        for i in range(imax):
            pbar.set_description(f"Now waiting for process with {all_parameters_iterated[i]}")
            pbar.update(1)
            # Tested : processes always run; no pause; the program waits here if necessary while further results are calculated
            results[i].get()
            
    
    
# Needed to avoid children process to create new children        
if __name__ == '__main__':
    try:
        os.mkdir("./Results")
    except:
        pass
    
    
    parallel_run()

In [ ]:
# Retrieve the results from the files

def change_type(key, value):
    if key in ['N', 'samples']:
        return int(value)
    
    if key in ["imprecision", 'brightness', 'purity', 'photon loss']:
        return float(value)
    
    else:
        # value must be a str
        return value




all_results_parallel = dict()

for directory in os.listdir("./Results"):
    if directory != ".ipynb_checkpoints":
        
        str_key = directory.split("_")
        
        keys = []
        
        for key_value in str_key:
            [key, value] =  key_value.split(":")
            value = change_type(key, value)
            keys.append(key)
            keys.append(value)
                
        keys = tuple(keys)
        all_results_parallel[keys] = []
        for file in os.listdir("./Results/" + directory):
            with open("./Results/" + directory + "/" + file, "rb") as f:
                all_results_parallel[keys].append(pickle.load(f))

In [ ]:
print(tuple_to_dict(list(all_results_parallel.keys())[0]))

## Results

In [ ]:
# Show the total number of computations in the dictionnary given 
def get_number_of_computation(results, x_axis, selection_dict):
    if x_axis == "photons":        
        n_min, n_max = get_photon_range(results, selection_dict)
        x_axis_key = "N"
    
    nb = (n_max - n_min + 1) * [0]
    for key in all_results_parallel.keys(): 
        dict_key = tuple_to_dict(key)
        if selection_dict.items() <= dict_key.items():
            nb[dict_key[x_axis_key] - n_min] = len(results[key])
    
    return nb


print("Total number of simulations :", sum(get_number_of_computation(all_results_parallel, "photons", {"method" : "post selection"})))

**Plot all solutions having loss < loss_max.** Do not put loss_max huge if there are many results.

In [ ]:
# Plot all approximations having a loss inferior to loss_max

loss_max = 0.015

use_imperfect_source = False
total_computation_count = 0

X_plot = np.linspace(range_min, range_max, 200)

simulator_backend = pcvl.BackendFactory().get_backend("SLOS")


def plot_all_solutions(solution_dictionnary, loss_max, legend = True):
    
    # Change the plot size
    default_figsize = mpl.rcParamsDefault['figure.figsize']
    mpl.rcParams['figure.figsize'] = [2 * value for value in default_figsize]
    
    
    global N
    global method

    for key in solution_dictionnary.keys():
        dict_key = tuple_to_dict(key)
        
        initiated = False
        
        for solution in solution_dictionnary[key]: 

            # Select good enough results (avoid graph compacting)
            if solution[5] <= loss_max:
                
                if not initiated:                
                    initiate(dict_key, display = False)
                    initiated = True

                plot_solution(X_plot, solution[0], solution[1], dict_key)


    plt.plot(X_plot, u(X_plot), 'k', label='Analytical solution')
    plt.title("Solutions with loss inferior to %0.3g" % loss_max)
    if legend:
        plt.legend()
    plt.show()
    
plot_all_solutions(all_results_parallel, loss_max, legend = True)

We then want to find for each key the best solution (in term of loss).

In [ ]:
best_solutions = get_best_solutions(all_results_parallel)
# print(best_solutions)


# We now plot all the best solutions

# It is possible to ask only one method
methods_to_plot = "all"
#methods_to_plot = ["pseudo PNR", "post selection"]

# Or the number of photons from which the curves have to be plot
min_photons = 6

default_figsize = mpl.rcParamsDefault['figure.figsize']
mpl.rcParams['figure.figsize'] = [2 * value for value in default_figsize]


for key in best_solutions.keys():
    dict_key = tuple_to_dict(key)
    
    if dict_key['N'] >= min_photons:

        if methods_to_plot == "all" or dict_key['method'] in methods_to_plot:
            
            best_index = best_solutions[key][1]
            solution = all_results_parallel[key][best_index]

            # Select good enough results (avoid graph compacting)

            initiate(dict_key, display = False)

            plot_solution(X_plot, solution[0], solution[1], dict_key)

plt.plot(X_plot, u(X_plot), 'k', label='Analytical solution')
plt.title(method)
plt.legend()
plt.show()

We now plot the boundary loss and the DE loss for the best solution in each category.

In [ ]:
# Small graphs
default_figsize = mpl.rcParamsDefault['figure.figsize']
mpl.rcParams['figure.figsize'] = [1 * value for value in default_figsize]

plot_decomposed_loss(all_results_parallel)

**Statistics**

We are now interested in finding the mean and the standard deviation for each set of parameters for both the selected computations and the global ones, relatively to the computation time, the number of iterations and the total number of computations. 

First with all computations.

In [ ]:
# Small graphs
default_figsize = mpl.rcParamsDefault['figure.figsize']
mpl.rcParams['figure.figsize'] = [1.5 * value for value in default_figsize]


methods = "all"
#methods = ["pseudo PNR", "post selection"]



plot_statistics(all_results_parallel, "loss", "photons")
plot_statistics(all_results_parallel, "computation time", "photons")
plot_statistics(all_results_parallel, "iteration number", "photons")
plot_statistics(all_results_parallel, "computation count", "photons")

Now with selections.

We can select the closest to the best approximation solutions so we can see what method gives us good results the more frequently.

In [ ]:
def select_good_approximations(results, epsilon, return_selected = False):
    # If return_selected == True, then all solutions having a loss less than epsilon*min(loss) are returned, as well as the best loss
    # Else, it returns the number of solutions having a loss less than epsilon*min(loss)
    
    best_solutions = get_best_solutions(results)
    
    best_of_all = min((best_solutions[key], key) for key in best_solutions.keys()) # Tuple ((loss, index), (method, N[, brightness]))
    
    good_approximations = dict()
    
    for key in best_solutions.keys():
        good_approximations[key] = []
        
        for solution in results[key]:
            if solution[5] <= best_of_all[0][0] * epsilon:
                good_approximations[key].append(solution)
    
    if return_selected:
        return good_approximations, best_of_all
    else:
        return sum([len(good_approximations[key]) for key in good_approximations.keys()])


In [ ]:
# Here we obtain for each parameters the proportion that is close enough to the best approximation.

methods = ["real solving", "fixed ponderations", "threshold", "post selection", "pseudo PNR"]


epsilon = 5  # > 1
# The plot doesn't always cover all the selected solutions, as there could be too many plots to do
plot_epsilon = min(2, epsilon)


# best : Tuple ((loss, index), (method, N[, brightness]))
good_approximations, best = select_good_approximations(all_results_parallel, epsilon, return_selected = True)
print("Minimal loss found : %0.3g" % best[0][0], "with a", best[1][0], "method")
print("Number of retained solutions :", sum([len(good_approximations[key]) for key in good_approximations.keys()]))

#plot_all_solutions(good_approximations, best[0][0] * plot_epsilon, legend = False)


# Change the plot size
default_figsize = mpl.rcParamsDefault['figure.figsize']
mpl.rcParams['figure.figsize'] = [0.5 * value for value in default_figsize]


x_axis = "photons"


for method in methods:
    proportion = np.array(get_number_of_computation(good_approximations, x_axis, {"method" : method})) \
    / np.array(get_number_of_computation(all_results_parallel, x_axis, {"method" : method}))
    
    
    if x_axis == "photons":
        n_min, n_max = get_photon_range(all_results_parallel, {"method" : method})
        X = range(n_min, n_max + 1)
    

    plt.bar(X, proportion);
    plt.ylim([0, 1.1])
    plt.xlabel(x_axis)
    plt.title("Proportion of solutions with " + str(epsilon) + " times the loss of the best with " + method)
    plt.grid(axis = 'y', linestyle = '--', linewidth = 0.7)
    plt.show()

In [ ]:
# Small graphs
default_figsize = mpl.rcParamsDefault['figure.figsize']
mpl.rcParams['figure.figsize'] = [1.5 * value for value in default_figsize]

methods = "all"
#methods = ["post selection", "pseudo PNR"]


plot_statistics(good_approximations, "loss", "photons", methods = methods, scale = "linear")
plot_statistics(good_approximations, "computation time", "photons", methods = methods)
plot_statistics(good_approximations, "iteration number", "photons", methods = methods)
plot_statistics(good_approximations, "computation count", "photons", methods = methods)

A last kind of selection could be interesting: we want to have the N best solutions for each parameter set.

In [ ]:
def select_N_good_approximations(N, results):
    # The N best solutions for each parameter set are returned
       
    good_N_approximations = dict()
    
    for key in best_solutions.keys():
        good_N_approximations[key] = []
        
        i = 0
        for solution in sorted(results[key], key = lambda sol: sol[5]):
            if i < N:
                good_N_approximations[key].append(solution)
            else:
                break
            i += 1
                
    return good_N_approximations
            

In [ ]:
N = 10

good_N_approximations = select_N_good_approximations(N, all_results_parallel)


# Small graphs
default_figsize = mpl.rcParamsDefault['figure.figsize']
mpl.rcParams['figure.figsize'] = [1.5 * value for value in default_figsize]


methods = "all"
#methods = ["post selection", "pseudo PNR"]



plot_statistics(good_N_approximations, "loss", "photons", methods = methods)
plot_statistics(good_N_approximations, "computation time", "photons", methods = methods)
plot_statistics(good_N_approximations, "iteration number", "photons", methods = methods)
plot_statistics(good_N_approximations, "computation count", "photons", methods = methods)

# TODO

## Parameters

Add the possibility to specify the reflexivity $R$ in the case of pseudo PNR and post selection.

Add the possibility to specify the penalisation coefficient $\alpha$ in the case of sampling.

Add the possibility to specify the order and the cut frequency in the case of sampling.

For all of these, add an easy way to implement parameters that are used only if there are particular other parameters.

## Computation

Find a solution to make the minimisation converging in case of sampling (tested : noise reduction on $f$)

Ideas : 
- Exact computation of $f'$ (difficulty : easy, but could require a lot of computing power)
- Exact computation of the gradient for the matrix parameters (difficulty : very hard)
- Exact computation of the gradient for the lambda coefficients (difficulty : medium)

The parameters of the noise reduction have to be better tested, or other methods for noise reduction could be used.
Also, the effect of imposing boundaries on $\lambda$ must be evaluated.

Add a parameter for the minimum number of modes in case of a post selection.
Maybe the way *pi_factors* is defined will not work anymore in this case and we would need to perform both the threshold and the pi_factors calculus.

Some results about derivation :

$$
loss = \sum_x F(f'(x), f(x), x)^2 + \alpha \lVert \lambda \rVert ^2 + \eta (f(x_0) - f_0)^2
$$

So for a particular $\lambda_i$,

$$
\frac{\partial loss}{\partial \lambda_i} = 2 \sum_x F(f'(x), f(x), x) \frac{\partial F(f'(x), f(x), x)}{\partial \lambda_i} + 2 \alpha \lambda_i + 2 \eta (f(x_0) - f_0) \frac{\partial f(0)}{\partial \lambda_i}
$$

And

$$
\frac{\partial F(f'(x), f(x), x)}{\partial \lambda_i} = \frac{\partial F}{\partial f'} \frac{\partial f'}{\partial \lambda_i} + \frac{\partial F}{\partial f} \frac{\partial f}{\partial \lambda_i} + \frac{\partial F}{\partial x} \frac{\partial x}{\partial \lambda_i}
$$

So 

$$
\frac{\partial F(f'(x), f(x), x)}{\partial \lambda_i} \approx \frac{\partial F}{\partial f'} \frac{p_i(x + \Delta x) - p_i(x - \Delta x)}{2 \Delta x}+ \frac{\partial F}{\partial f} p_i(x)
$$

Furthermore,
$$
\frac{\partial f(x_0)}{\partial \lambda_i} = p_i(x_0)
$$

And in our case, $\frac{\partial F}{\partial f'} = 1$

and $\frac{\partial F}{\partial f} = \lambda (\kappa + \tan (\lambda x))$

## Generalisation

Generalise to all kind of DE:
- multivariable
- vector outputs
- More orders

## Visualisation

Make the plot_statistics function work with log scale